In [1]:
from dotenv import load_dotenv
import requests
from requests import get
import json
import base64
import os

# Entrada a la API

In [2]:
load_dotenv()

client_id = os.getenv('client_id')
client_secret = os.getenv('client_secret')

In [3]:
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials
client_id = client_id
client_secret = client_secret
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 
import requests
import pandas as pd
import time 

In [4]:
# Getting Access
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [5]:
# Saving header information for access
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [6]:
# Data Extraction

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

# your artist ID - The Weeknd
artist_id = '1Xyo4u8uXC1ZmMpatF05PJ'

# extract all albums
r = requests.get(BASE_URL + 'artists/' + artist_id + '/albums', 
                 headers=headers, 
                 params={'include_groups': 'album', 'limit': 50})
d = r.json()

In [7]:
# 1. Extract all tracks and track information from all albums

trackinfo = []   # will hold all track info
albums = [] # to keep track of duplicates

# loop over albums and get all tracks
for album in d['items']:
    album_name = album['name']

    # here's a hacky way to skip over albums we've already grabbed
    trim_name = album_name.split('(')[0].strip()
    if trim_name.upper() in albums:
        continue
    albums.append(trim_name.upper()) # use upper() to standardize
    
    # this takes a few seconds so let's keep track of progress    
    print(album_name)
    
    # pull all tracks from this album
    r = requests.get(BASE_URL + 'albums/' + album['id'] + '/tracks', 
        headers=headers)
    tracks = r.json()['items']
    
    for track in tracks:
        # get track information
        f = requests.get(BASE_URL + 'tracks/' + track['id'], 
            headers=headers)
        f = f.json()
    
     # combine with album info
        f.update({
            'album_name': album_name,
            'release_date': album['release_date'],
            'album_id': album['id']
        })
        
        trackinfo.append(f)

Starboy (Deluxe)
Live At SoFi Stadium
Avatar: The Way of Water (Original Motion Picture Soundtrack)
Dawn FM (Alternate World)
After Hours (Deluxe)
My Dear Melancholy,
Beauty Behind The Madness
Kiss Land (Deluxe)
Trilogy (Original Version)
Echoes Of Silence (Original)
Thursday (Original)
House Of Balloons (Original)


In [8]:
df = pd.DataFrame(trackinfo)
df.head()

,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri,album_name,release_date,album_id
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,230461,True,{'isrc': 'USUG11600976'},{'spotify': 'https://open.spotify.com/track/76...,https://api.spotify.com/v1/tracks/76yLBykniz0S...,76yLBykniz0SSNv0jj17M2,False,Starboy (feat. Daft Punk),62,None,1,track,spotify:track:76yLBykniz0SSNv0jj17M2,Starboy (Deluxe),2023-03-14,35dut3ICqF3NEDkjxfzJJ1
1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,249207,True,{'isrc': 'USUG11600997'},{'spotify': 'https://open.spotify.com/track/6f...,https://api.spotify.com/v1/tracks/6fSvz9yrQY40...,6fSvz9yrQY407xmr7cdjuA,False,Party Monster,60,None,2,track,spotify:track:6fSvz9yrQY407xmr7cdjuA,Starboy (Deluxe),2023-03-14,35dut3ICqF3NEDkjxfzJJ1
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,220308,False,{'isrc': 'USUG11600975'},{'spotify': 'https://open.spotify.com/track/4k...,https://api.spotify.com/v1/tracks/4k47x4UkxG21...,4k47x4UkxG215nFLMFna8A,False,False Alarm,56,None,3,track,spotify:track:4k47x4UkxG215nFLMFna8A,Starboy (Deluxe),2023-03-14,35dut3ICqF3NEDkjxfzJJ1
3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,218879,True,{'isrc': 'USUG11600998'},{'spotify': 'https://open.spotify.com/track/4z...,https://api.spotify.com/v1/tracks/4zHa39io9pjs...,4zHa39io9pjsiAE6LwM2NH,False,Reminder,59,None,4,track,spotify:track:4zHa39io9pjsiAE6LwM2NH,Starboy (Deluxe),2023-03-14,35dut3ICqF3NEDkjxfzJJ1
4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,232881,True,{'isrc': 'USUG11600999'},{'spotify': 'https://open.spotify.com/track/3m...,https://api.spotify.com/v1/tracks/3mynPhglTz2G...,3mynPhglTz2Ggh8SxA34kR,False,Rockin’,55,None,5,track,spotify:track:3mynPhglTz2Ggh8SxA34kR,Starboy (Deluxe),2023-03-14,35dut3ICqF3NEDkjxfzJJ1
